In [55]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import re

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import fasttext
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from code_utils.utils import preprocess

In [2]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_model.json')

In [6]:
df_not_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_not_ipcc.jsonl', lines= True)

In [7]:
df_ipcc=df_ipcc[['doi', 'title_OA','topics']]

In [8]:
df_not_ipcc=df_not_ipcc[['doi', 'title_OA','topics']]

In [9]:
df_ipcc['label']="ipcc_like"
df_not_ipcc['label']="not_ipcc_like"

In [10]:
df_ipcc=df_ipcc.drop_duplicates(subset=['doi'])

In [11]:
len(df_ipcc)

54528

In [12]:
len(df_not_ipcc)

54532

In [33]:
df_ipcc=df_ipcc[pd.isna(df_ipcc.topics)==False]

In [34]:
df_not_ipcc=df_not_ipcc.iloc[:54148,:]

In [45]:
df_all=pd.concat([df_ipcc,df_not_ipcc]).reset_index()
del df_all['index']

In [46]:
df_all["topics_name"]=df_all["topics"].apply(lambda x: ''.join([y.get('name') for y in x]))

In [48]:
df_all["title_topics"]=df_all["title_OA"]+" "+df_all["topics_name"]


In [49]:
df_all['label'] = '__label__' + df_all['label'].astype(str)

In [57]:
df_all['category_description'] = df_all['label'] + ' ' + df_all['title_topics']
df_all.loc[:,'category_description']=df_all.loc[:,'category_description'].apply(lambda x: str(x).lower())

In [58]:
train, test = train_test_split(df_all, test_size=0.2, stratify=df_all["label"])

In [60]:
test["label"].value_counts(normalize=True)

label
__label__ipcc_like        0.5
__label__not_ipcc_like    0.5
Name: proportion, dtype: float64

In [61]:
train.to_csv("ipcc.train", columns=["category_description"], index=False, header=False)
test.to_csv("ipcc.test", columns=["category_description"], index=False, header=False)

In [62]:
model = fasttext.train_supervised(input="ipcc.train", epoch=5)
model.test("ipcc.test")

(18117, 0.9490533752828835, 0.9490533752828835)